In [1]:
from PIL import Image
import glob
import os
import numpy as np
%matplotlib inline

import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.utils import shuffle

In [2]:
from scipy.misc import imresize
import cv2

In [3]:
driving_log = pd.read_csv('data/data/driving_log.csv')

In [5]:
center_images, right_images, left_images = driving_log['center'], driving_log['right'], driving_log['left']

In [6]:
data_images = (pd.concat([right_images, center_images]))
data_images = np.array(pd.concat([data_images, left_images]))

In [7]:
len(data_images)

24108

In [8]:
def flip_the_image(img, steer):
    seed = np.random.randint(2)
    if seed == 1:
        img = np.fliplr(img)
        steer = -steer
    return img, steer

In [9]:
def colorspace(img):
    return cv2.cvtColor(img, cv2.COLOR_RGB2HSV)

In [10]:
row, col = 16, 32

In [11]:
def img_resize(img):
    w, h = img.size
    img = img.crop((0, h*0.3, w, h-22))
    image_resize = imresize(img, (row, col))
    image = np.asarray(image_resize)
    return image

In [12]:
def brightness(img):
    seed = np.random.randint(2)
    if seed == 1:
        img = cv2.cvtColor(img, cv2.COLOR_RGB2HSV)
        random_bright = np.random.uniform(0.5,0.9)
        img[:,:,2] = img[:,:,2] * random_bright
        img = cv2.cvtColor(img, cv2.COLOR_HSV2RGB)
    return img

In [13]:
steering_center, steering_right, steering_left = driving_log['steering'],  (driving_log['steering'] - 0.15), (driving_log['steering'] + 0.15)

In [14]:
steering_angles = (pd.concat([steering_right, steering_center]))
steering_angles = np.array(pd.concat([steering_angles, steering_left]))

In [15]:
len(steering_angles)

24108

In [16]:
def downsampling(steer, down_threshold):
    
#down_threshold = 0.8
    sample = []
    for i in range(len(steer)):
        if abs(steer[i]) < 0.1:
        #print (1)
            if np.random.uniform(0,1) < down_threshold:
                sample.append(i)
        
        else:
            sample.append(i)
    return sample

In [17]:
len(steering_angles)

24108

In [18]:
len(downsampling(steering_angles, 0.75))

21987

In [20]:
X_train, y_train = shuffle(data_images, steering_angles)
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, random_state=0, test_size=0.1)

In [22]:
def generator(batch_size, data, labels):
    batch_images = np.zeros(shape=(batch_size,row,col,3), dtype=np.float32)
    batch_labels = np.zeros(shape=(batch_size), dtype=np.float32)
    #batch_size = 100
    while True:
        down_sample = downsampling(labels, 0.75)
        steer_downsampled = labels[down_sample]
        data_downsampled = data[down_sample]
        shuffle = np.random.randint(len(down_sample), size= len(down_sample))
        image_files = data_downsampled[shuffle]
        steering = steer_downsampled[shuffle] 
        #np.random.shuffle(image_files)
        for i in range(batch_size):
            img_path =  os.path.join('data/data/{}'.format(image_files[i]))#.format(i)
            img_path = ''.join(img_path.split())
            image = Image.open(img_path)
            steer = steering[i]
            # some pre-processing with the image
            image = img_resize(image)
            image, steer = flip_the_image(image, steer)
            image = brightness(image)
            batch_labels[i] = steer
            #batch_images.append(image)
            batch_images[i] = image
        #batch_images = np.array(batch_images)
        yield batch_images, batch_labels

In [26]:
train_gen = (generator(128, X_train, y_train))

In [27]:
valid_gen = (generator(128, X_val, y_val))

In [30]:
from keras.models import Sequential
from keras.layers import Dense, Input, Activation
from keras.layers import Conv2D, Flatten, Dropout
from keras.optimizers import Adam
from keras.regularizers import l2, activity_l2

Using TensorFlow backend.


In [31]:
from keras.layers.core import Lambda
from keras.layers.pooling import MaxPooling2D

In [32]:
model = Sequential()
model.add(Lambda(lambda x: x / 255 - 0.5, input_shape=(row, col, 3)))

model.add(Conv2D(8, 3, 3, border_mode='valid', activation='relu'))
model.add(MaxPooling2D(pool_size=(3, 3)))
model.add(Dropout(0.5))
model.add(Flatten())
model.add(Dense(1)) # there should be no activation function
                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                        
model.summary()


____________________________________________________________________________________________________
Layer (type)                     Output Shape          Param #     Connected to                     
lambda_1 (Lambda)                (None, 16, 32, 3)     0           lambda_input_1[0][0]             
____________________________________________________________________________________________________
convolution2d_1 (Convolution2D)  (None, 14, 30, 8)     224         lambda_1[0][0]                   
____________________________________________________________________________________________________
maxpooling2d_1 (MaxPooling2D)    (None, 4, 10, 8)      0           convolution2d_1[0][0]            
____________________________________________________________________________________________________
dropout_1 (Dropout)              (None, 4, 10, 8)      0           maxpooling2d_1[0][0]             
___________________________________________________________________________________________

In [33]:
adam = Adam(lr=0.001)
model.compile(loss='mse',
              optimizer= adam,
              metrics=['mean_absolute_error'])

In [38]:
history = model.fit_generator(train_gen, nb_epoch =35,
                             samples_per_epoch=20000,verbose=1,
                              validation_data=valid_gen,nb_val_samples=5000)

Epoch 1/35
19968/20000 [============================>.] - ETA: 0s - loss: 0.0149 - mean_absolute_error: 0.0915

/home/rahul/anaconda3/lib/python3.5/site-packages/keras/engine/training.py:1527: UserWarning: Epoch comprised more than `samples_per_epoch` samples, which might affect learning results. Set `samples_per_epoch` correctly to avoid this warning.
  warnings.warn('Epoch comprised more than '


20096/20000 [==============================] - 47s - loss: 0.0149 - mean_absolute_error: 0.0914 - val_loss: 0.0142 - val_mean_absolute_error: 0.0860
Epoch 2/35
20096/20000 [==============================] - 45s - loss: 0.0149 - mean_absolute_error: 0.0912 - val_loss: 0.0142 - val_mean_absolute_error: 0.0860
Epoch 3/35
20096/20000 [==============================] - 45s - loss: 0.0151 - mean_absolute_error: 0.0915 - val_loss: 0.0142 - val_mean_absolute_error: 0.0879
Epoch 4/35
20096/20000 [==============================] - 44s - loss: 0.0151 - mean_absolute_error: 0.0911 - val_loss: 0.0137 - val_mean_absolute_error: 0.0862
Epoch 5/35
20096/20000 [==============================] - 44s - loss: 0.0151 - mean_absolute_error: 0.0918 - val_loss: 0.0154 - val_mean_absolute_error: 0.0884
Epoch 6/35
20096/20000 [==============================] - 44s - loss: 0.0149 - mean_absolute_error: 0.0912 - val_loss: 0.0144 - val_mean_absolute_error: 0.0877
Epoch 7/35
20096/20000 [===========================

In [39]:
# serialize model to JSON
model_json = model.to_json()
with open("model.json", "w") as json_file:
    json_file.write(model_json)
# serialize weights to HDF5
model.save_weights("model.h5")
print("Saved model to disk")

Saved model to disk
